# Cervical Lesion Classification with Detectron2

## Description

This notebook aims to develop a deep learning model using the Mask R-CNN architecture from Detectron2 for the **classification and detection of cervical lesions** in biomedical images. The process begins with annotations previously generated in **COCO JSON** or **VGG VIA JSON** format. It includes the unification and conversion of annotations, dataset registration in Detectron2, sample visualization, model training, and performance evaluation.

Detectron2 provides a robust and highly optimized framework for instance segmentation and object detection, and its application in this context seeks to automate and improve the precision of colposcopic diagnosis.

## Authors

- **Ramiro Israel Vivanco Gualán**, MSc  
  Department of Chemistry and Exact Sciences, Universidad Técnica Particular de Loja (UTPL), Ecuador

- **Yuliana Jiménez-Gaona**, MSc  
  Department of Chemistry and Exact Sciences, UTPL, Ecuador

- **Bernardo Vega-Crespo**, MD, PhD  
  Faculty of Medical Sciences, Universidad de Cuenca, Ecuador

- **Verónica Muñoz**, MD  
  Faculty of Medical Sciences, Universidad de Cuenca, Ecuador

- **Veronique Verhoeven**, MD, PhD  
  University of Antwerp, Belgium

---

## Objectives

- Unify annotations from various sources (COCO, VGG VIA) into a single standard file.
- Register custom datasets in Detectron2 for training and validation.
- Train a Mask R-CNN model from the `model_zoo` for lesion classification and segmentation.
- Evaluate the model's performance in terms of class-wise detection and segmentation accuracy.
- Export segmentation results, predictions, and evaluation metrics in both visual and numerical formats.

---

## Dataset

- **Original formats**: COCO JSON, VGG VIA JSON.
- **Expected labels**: `demy`, `isch`, among other lesion-related classes.
- **Source**: Biomedical cervix images from clinical studies (e.g., Intel dataset, KAIME).

---

## Tools and Libraries

- Jupyter Notebook / Google Colab
- Detectron2 (Facebook AI Research)
- OpenCV, Matplotlib, Pandas
- PyTorch (CUDA enabled)
- makesense.ai / VGG VIA (for annotation purposes)


In [3]:
!pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 5.1 MB/s eta 0:00:00:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 34.5 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.9 MB/s eta 0:00:0000:010:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.3 MB/s eta 0:00:0000:010:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.4 MB/s eta 0:00:00:00:010:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 10.9 MB/s eta 0:00:0000:010:01
INFO: pip is still looking at multiple versions of torchvision to determine wh

In [ ]:
# =======================
# File and Data Handling
# =======================
import os                              # File system operations
import json                            # JSON file manipulation
import uuid                            # Unique identifiers
from glob import glob                  # File search using patterns
from pathlib import Path               # Object-oriented filesystem paths
import warnings                        # Suppress warning messages
import h5py                            # HDF5 file I/O for saving models
import joblib                          # For saving trained models
import random                          # Random number generation
from collections import Counter        # Frequency counting

# =====================
# Numerical Processing
# =====================
import numpy as np                     # Numerical array operations
import pandas as pd                    # DataFrame structures and CSV handling

# ====================
# Image Processing
# ====================
import cv2                             # OpenCV for image handling
from PIL import Image                  # PIL for image manipulation

# ===================
# Data Visualization
# ===================
import matplotlib.pyplot as plt        # Plotting and data visualization
import seaborn as sns                  # Advanced plotting (heatmaps, distributions)

# ============================
# Machine Learning Utilities
# ============================
from sklearn.model_selection import train_test_split  # Train/test split

# ============================
# Deep Learning (TensorFlow)
# ============================
import tensorflow as tf
from tensorflow.keras import layers, Model             # Model architecture
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU           # Segmentation metric

# ===================
# Detectron2 Framework
# ===================
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
    build_detection_test_loader,
)
from detectron2.data.datasets import (
    register_coco_instances,
    coco,
    load_coco_json,
)
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.evaluation import (
    COCOEvaluator,
    inference_on_dataset,
)
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# ===================
# Memory Management
# ===================
import gc
gc.collect()
torch.cuda.empty_cache()

# ===================
# Progress Monitoring
# ===================
from tqdm import tqdm                  # Progress bar for loops


## Actual Paths

In [ ]:
# Real paths
BASE_DIR = Path.cwd()
INTEL_DIR = BASE_DIR / "ColpoToolLab" / "intel"
JSON_DIR = BASE_DIR / "ColpoToolLab" / "detectron_data"

print(f"📂 INTEL_DIR: {INTEL_DIR}")

## JSON Files to Use

In [ ]:
# 🧾 JSONs to use
json_paths = [
    JSON_DIR / "colpo_2_coco.json",
    JSON_DIR / "coco_labels_32_aw_m_p_av_2025-04-10-07-28-25.json"
]

## List Real Images in Intel Folder

In [ ]:
# Step 1: List real images in Intel folder
imagenes_intel = {img.name for img in INTEL_DIR.glob("*.jpg")}
print(f"🖼️ Total .jpg images in Intel: {len(imagenes_intel)}")
print(f"🔍 Examples: {list(imagenes_intel)[:5]}")

## Load JSONs and Filter Valid Images

In [ ]:
# Step 2: Load JSONs and filter valid images
imagenes_validas = {}
anotaciones_validas = []

for json_file in json_paths:
    print(f"\n📦 Processing: {json_file.name}")
    if not json_file.exists():
        print(f"❌ File not found: {json_file}")
        continue

    with open(json_file, 'r') as f:
        data = json.load(f)

    images_json = data.get("images", [])
    annotations_json = data.get("annotations", [])

    print(f"📷 Images in JSON: {len(images_json)} | 🧩 Annotations: {len(annotations_json)}")

    # Validate images that physically exist
    for img in images_json:
        nombre = Path(img["file_name"]).name
        ruta_imagen = INTEL_DIR / nombre

        if ruta_imagen.exists():
            imagenes_validas[img["id"]] = nombre
        else:
            print(f"⚠️ Missing image on disk: {nombre}")

    # Validate annotations whose image exists
    for ann in annotations_json:
        if ann["image_id"] in imagenes_validas:
            nombre_img = imagenes_validas[ann["image_id"]]
            ruta_img = INTEL_DIR / nombre_img

            if ruta_img.exists():  # Re-confirmation (double check)
                segmentacion = ann.get("segmentation", [])
                if isinstance(segmentacion, list) and len(segmentacion) > 0 and len(segmentacion[0]) >= 6:
                    anotaciones_validas.append({
                        "nombre": nombre_img,
                        "categoria_id": ann["category_id"],
                        "segmentacion": segmentacion
                    })
                else:
                    print(f"⚠️ Invalid annotation (empty or short segmentation): {nombre_img}")


## Count by Category

In [ ]:
# 📊 Step 3: Count by category
conteo = Counter(ann["categoria_id"] for ann in anotaciones_validas)

In [ ]:
# 🧾 Results
print(f"\n✅ Total valid images (with at least one valid annotation): {len(set(ann['nombre'] for ann in anotaciones_validas))}")
print(f"✅ Total valid annotations: {len(anotaciones_validas)}")

## Split into Train and Val Sets

In [ ]:
# 🧪 Extra step: split into train and val
from sklearn.model_selection import train_test_split

# Get unique image names
imagenes_unicas = sorted(list(set(ann["nombre"] for ann in anotaciones_validas)))

# Split into 80% train, 20% val
train_imgs, val_imgs = train_test_split(imagenes_unicas, test_size=0.2, random_state=42)

# Assign each annotation to its set
anotaciones_train = [a for a in anotaciones_validas if a["nombre"] in train_imgs]
anotaciones_val = [a for a in anotaciones_validas if a["nombre"] in val_imgs]


## Class Count & Sample

In [ ]:
# 📊 Class count
print("\n📊 Count per class:")
for clase, cantidad in conteo.items():
    print(f" - Class {clase}: {cantidad}")


In [ ]:
# 🧪 Sample
print("\n🧪 Sample of 5 annotations:")
for ann in anotaciones_validas[:5]:
    print(f" - Image: {ann['nombre']}, Class: {ann['categoria_id']}, Segment: {str(ann['segmentacion'])[:40]}...")


## Visualization

In [ ]:
# 📈 Visualization
if not conteo:
    print("⚠️ No data to plot.")
else:
    clases = list(conteo.keys())
    cantidades = list(conteo.values())

    plt.figure(figsize=(8, 5))
    plt.bar(clases, cantidades, color='cornflowerblue', edgecolor='black')
    plt.xlabel("Class ID")
    plt.ylabel("Number of Annotations")
    plt.title("Class Distribution in Valid Annotations")
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()

## Initialization and Data Formatting for Detectron2

In [ ]:
# ✅ Initialization
final_images = []
final_annotations = []
img_id_map = {}
img_id_counter = 1
ann_id_counter = 1

for ann in anotaciones_validas:
    nombre = ann["nombre"]
    segmentacion = ann["segmentacion"]

    # ❌ Skip empty or invalid annotations
    if not isinstance(segmentacion, list) or len(segmentacion) == 0 or len(segmentacion[0]) < 6:
        print(f"⚠️ Invalid annotation skipped: {nombre}")
        continue

    # 🖼️ Process image if not already registered
    if nombre not in img_id_map:
        img_path = str(INTEL_DIR / nombre)

        imagen = cv2.imread(img_path)
        if imagen is None:
            print(f"⚠️ Image not found or corrupted: {nombre}")
            continue
        height, width = imagen.shape[:2]  # OpenCV gives (height, width)

        final_images.append({
            "id": img_id_counter,
            "file_name": nombre,
            "height": height,
            "width": width
        })
        img_id_map[nombre] = img_id_counter
        img_id_counter += 1

    # 📦 Calculate bbox and area
    puntos = segmentacion[0]
    x_coords = puntos[0::2]
    y_coords = puntos[1::2]
    if not x_coords or not y_coords:
        print(f"⚠️ Empty coordinates, skipped: {nombre}")
        continue
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    bbox = [x_min, y_min, x_max - x_min, y_max - y_min]
    area = bbox[2] * bbox[3]

    final_annotations.append({
        "id": ann_id_counter,
        "image_id": img_id_map[nombre],
        "category_id": ann["categoria_id"],
        "segmentation": segmentacion,
        "iscrowd": 0,
        "bbox": bbox,
        "area": area
    })
    ann_id_counter += 1

## Define Categories

In [ ]:
# ✅ Categories
final_categories = [
    {"id": 1, "name": "aw"},
    {"id": 2, "name": "m"},
    {"id": 3, "name": "p"},
    {"id": 4, "name": "av"},
    {"id": 5, "name": "y"}
]

## Dataset Construction for Detectron2

In [ ]:
from PIL import Image

def get_colpo_dicts_train():
    return construir_dataset(anotaciones_train)

def get_colpo_dicts_val():
    return construir_dataset(anotaciones_val)

def construir_dataset(anotaciones_subset):
    dataset_dicts = []
    nombres_procesados = set()
    img_id_counter = 0  # Incremental counter for consistent IDs

    for ann in anotaciones_subset:
        nombre = ann["nombre"]
        img_path = INTEL_DIR / nombre
        if not img_path.exists() or nombre in nombres_procesados:
            continue

        imagen = cv2.imread(str(img_path))
        if imagen is None:
            continue

        height, width = imagen.shape[:2]
        record = {
            "file_name": str(img_path),
            "image_id": img_id_counter,  # Consistent ID, not variable like hash()
            "height": height,
            "width": width,
            "annotations": []
        }

        for sub_ann in [a for a in anotaciones_subset if a["nombre"] == nombre]:
            puntos = sub_ann["segmentacion"][0]

            # Validate out-of-range points
            if max(puntos) > 1.5 * max(width, height):
                print(f"⚠️ Out-of-scale coordinates in {nombre} → max point: {max(puntos)} | Size: {width}x{height}")

            x_coords = puntos[0::2]
            y_coords = puntos[1::2]
            bbox = [min(x_coords), min(y_coords), max(x_coords) - min(x_coords), max(y_coords) - min(y_coords)]

            record["annotations"].append({
                "bbox": bbox,
                "bbox_mode": 0,  # BoxMode.XYWH_ABS
                "segmentation": [puntos],
                "category_id": sub_ann["categoria_id"] - 1,
                "iscrowd": 0
            })

        dataset_dicts.append(record)
        nombres_procesados.add(nombre)
        img_id_counter += 1  # Increment counter per registered image

    return dataset_dicts

## Data Integrity Checks

In [ ]:
print("🔎 Checking malformed segmentations...")

errores_seg = 0
for d in get_colpo_dicts_train():
    for ann in d["annotations"]:
        if len(ann["segmentation"][0]) < 6:
            print(f"❌ Invalid segmentation in {d['file_name']} with category {ann['category_id']}")
            errores_seg += 1

print(f"✅ Verification complete. Total invalid segmentations: {errores_seg}")

print("🔎 Checking category IDs...")
ids = [cat["id"] for cat in final_categories]
if sorted(ids) != list(range(len(final_categories))):
    print("❌ Category IDs are not sequential from 0. This may cause errors.")
else:
    print("✅ Category IDs are correctly defined.")

## Registering Datasets in Detectron2

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# 🔁 Remove if already registered
for dname in ["colpo_train", "colpo_val"]:
    if dname in DatasetCatalog.list():
        DatasetCatalog.remove(dname)

# ✅ Re-register
DatasetCatalog.register("colpo_train", get_colpo_dicts_train)
DatasetCatalog.register("colpo_val", get_colpo_dicts_val)

MetadataCatalog.get("colpo_train").set(thing_classes=[c["name"] for c in final_categories])
MetadataCatalog.get("colpo_val").set(thing_classes=[c["name"] for c in final_categories])

print("✅ Datasets registered: colpo_train and colpo_val.")

MetadataCatalog.get("colpo_train").set(thing_classes=[cat["name"] for cat in final_categories])
print("✅ Dataset in memory registered as 'colpo_train'.")


## Retrieve a Sample from the Training Dataset

In [ ]:
# ✅ Retrieve a sample from the training dataset
muestra = get_colpo_dicts_train()[0]
print(f"🔎 Image: {muestra['file_name']}")
print(f"✅ Width: {muestra['width']} | Height: {muestra['height']}")

from PIL import Image
img = Image.open(muestra["file_name"])
print(f"🧾 Actual size: {img.size}")  # Should match width and height


In [ ]:
# 🔄 Re-register datasets (in case you need to force cleanup or reload)
DatasetCatalog.remove("colpo_train")
DatasetCatalog.remove("colpo_val")

DatasetCatalog.register("colpo_train", get_colpo_dicts_train)
DatasetCatalog.register("colpo_val", get_colpo_dicts_val)

MetadataCatalog.get("colpo_train").set(thing_classes=[cat["name"] for cat in final_categories])
MetadataCatalog.get("colpo_val").set(thing_classes=[cat["name"] for cat in final_categories])

print("✅ Datasets updated and re-registered: colpo_train and colpo_val.")

## Visualize Random Samples from the Dataset

In [ ]:
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer

def visualizar_muestras(dataset_dicts, metadata, num_muestras=5, titulo="Samples"):
    plt.figure(figsize=(15, 5 * num_muestras))
    count = 0
    for d in random.sample(dataset_dicts, min(num_muestras, len(dataset_dicts))):
        img_path = d["file_name"]
        img = cv2.imread(img_path)

        if img is None:
            print(f"❌ Image not found: {img_path}")
            continue

        visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        plt.subplot(num_muestras, 1, count + 1)
        plt.imshow(vis.get_image()[:, :, ::-1])
        plt.title(f"{titulo} #{count+1}")
        plt.axis('off')
        count += 1

    if count == 0:
        print("⚠️ Samples could not be visualized (all failed).")
    else:
        plt.tight_layout()
        plt.show()

## Directory and Image Count

In [ ]:
from pathlib import Path

print("📁 INTEL_DIR:", INTEL_DIR)
print("🖼️ Total images:", len(list(INTEL_DIR.glob("*.jpg"))))


## Execute Sample Visualization

In [ ]:
# Execute visualization
visualizar_muestras(
    DatasetCatalog.get("colpo_train"),
    MetadataCatalog.get("colpo_train"),
    num_muestras=5,
    titulo="Visualization - Full Dataset"
)

## Display Random Examples with Real Segmentations

In [ ]:
from detectron2.utils.visualizer import ColorMode

print("🖼️ Displaying 3 random examples with real segmentations...")

for d in random.sample(get_colpo_dicts_train(), 3):
    img = cv2.imread(d["file_name"])
    v = Visualizer(
        img[:, :, ::-1],
        metadata=MetadataCatalog.get("colpo_train"),
        scale=0.5,
        instance_mode=ColorMode.IMAGE_BW  # Grayscale background for emphasis
    )
    v = v.draw_dataset_dict(d)
    plt.figure(figsize=(8, 6))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.title(d["file_name"])
    plt.axis('off')
    plt.show()

## Free Memory (Optional but Useful if Coming from a Previous Session)

In [ ]:
def get_gpu_with_max_free_memory():
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        free_memories = [int(x) for x in result.strip().split('\n')]
        best_gpu = free_memories.index(max(free_memories))
        print(f"🧠 Selecting GPU {best_gpu} with {free_memories[best_gpu]} MiB free.")
        return best_gpu
    except Exception as e:
        print(f"❌ Error while querying GPU: {e}")
        return 0

# Set the visible GPU
best_gpu_id = get_gpu_with_max_free_memory()
os.environ["CUDA_VISIBLE_DEVICES"] = str(best_gpu_id)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# 🔁 Remove datasets if already registered
for dname in ["colpo_train", "colpo_val"]:
    if dname in DatasetCatalog.list():
        DatasetCatalog.remove(dname)

# ✅ Re-register
DatasetCatalog.register("colpo_train", get_colpo_dicts_train)
DatasetCatalog.register("colpo_val", get_colpo_dicts_val)

MetadataCatalog.get("colpo_train").set(thing_classes=[c["name"] for c in final_categories])
MetadataCatalog.get("colpo_val").set(thing_classes=[c["name"] for c in final_categories])

print("✅ Datasets registered: colpo_train and colpo_val.")

## Training Configuration and Model Setup 

In [ ]:
# 📁 Output directory for trained models
OUTPUT_DIR = "output/colpo_model"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ⚙️ Model configuration
cfg = get_cfg()
cfg.OUTPUT_DIR = OUTPUT_DIR

# 🧠 Use base model architecture
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

# 📦 Load only backbone weights (without pretrained head)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# 📊 Custom datasets
cfg.DATASETS.TRAIN = ("colpo_train",)
cfg.DATASETS.TEST = ("colpo_val",)

# 🎯 Custom classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.NAME = "StandardROIHeads"  # 👈 Avoid conflict with COCO head

# 🧪 Score threshold for predictions
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05

# 🚀 Device: GPU
cfg.MODEL.DEVICE = "cuda:0"

# 🧮 Optimizer hyperparameters
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 3000  # 👈 Increased to ensure actual learning
cfg.SOLVER.STEPS = []  # No learning rate decay
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256

# 🧪 Evaluation frequency
cfg.TEST.EVAL_PERIOD = 500

# ✅ Train from scratch (no resume)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


## CONFIGURATION WITH TRAINED CHECKPOINT

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.OUTPUT_DIR = "output/colpo_model"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("colpo_val",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
cfg.MODEL.DEVICE = "cuda:0"

# Register classes
MetadataCatalog.get("colpo_val").thing_classes = ["aw", "m", "p", "av", "y"]
predictor = DefaultPredictor(cfg)

## EVALUATION 

In [ ]:
evaluator = COCOEvaluator("colpo_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "colpo_val")
results = inference_on_dataset(predictor.model, val_loader, evaluator)
print("\n📊 Evaluation results:")
print(results)


##  CLASS-WISE AP (Manual Computation)

In [ ]:
COCO_JSON_PATH = Path("output/colpo_model/colpo_val_coco_format.json")
coco_gt = COCO(str(COCO_JSON_PATH))
coco_dt = coco_gt.loadRes("output/colpo_model/coco_instances_results.json")
coco_eval = COCOeval(coco_gt, coco_dt, iouType="segm")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

cats = coco_gt.loadCats(coco_gt.getCatIds())
class_names = [c["name"] for c in cats]
ap_results = []
for idx, name in enumerate(class_names):
    precision = coco_eval.eval["precision"][:, :, idx, 0, 0]
    precision = precision[precision > -1]
    ap = np.mean(precision) if precision.size else float("nan")
    ap_results.append((name, ap))


## HEATMAP

In [ ]:
df_ap = pd.DataFrame(ap_results, columns=["Class", "AP"]).dropna()
sns.heatmap(df_ap.set_index("Class").T, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("📊 AP per Class (manually calculated)")
plt.show()

## VISUALIZATION

In [ ]:
dataset_val = DatasetCatalog.get("colpo_val")
for d in random.sample(dataset_val, 5):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   MetadataCatalog.get("colpo_val"),
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Prediction", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()


## CONFIGURATION REVIEW FUNCTION

In [ ]:
def revisar_configuracion(cfg):
    print("\n🧠 CONFIGURATION REVIEW")
    print("📦 Weights loaded from:", cfg.MODEL.WEIGHTS)
    print("📊 Number of classes:", cfg.MODEL.ROI_HEADS.NUM_CLASSES)
    print("📁 VAL dataset:", cfg.DATASETS.TEST)
    print("🎯 Score threshold:", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
    print("📎 Registered categories:", MetadataCatalog.get(cfg.DATASETS.TEST[0]).thing_classes)
    if cfg.MODEL.ROI_HEADS.NUM_CLASSES != len(MetadataCatalog.get(cfg.DATASETS.TEST[0]).thing_classes):
        print("❌ MISMATCH between NUM_CLASSES and registered categories. Check metadata registration.")
    else:
        print("✅ Number of classes and categories are aligned.")

revisar_configuracion(cfg)


## Generate a consistent mapping of image names to IDs

In [ ]:
import json
from pathlib import Path
from detectron2.data import DatasetCatalog, MetadataCatalog

# Paths
COCO_JSON_PATH = Path("output/colpo_model/colpo_val_coco_format.json")
VAL_IMG_DIR = Path("/tf/workspace/ColpoToolLab/intel")

# 🔁 Build a consistent map between image names and IDs
# ⚠️ Reuse this also when constructing the COCO JSON annotation section
print("\n🔧 Generating img_id_map from unique names in the 'colpo_val' dataset...")
dataset_val = DatasetCatalog.get("colpo_val")

nombres_imagenes = sorted(list(set([Path(d["file_name"]).name for d in dataset_val])))
img_id_map = {nombre: i for i, nombre in enumerate(nombres_imagenes)}

# Show a sample of the generated map
print("🗺️ Sample of img_id_map (first 5):")
for k in list(img_id_map.keys())[:5]:
    print(f"  {k} → ID {img_id_map[k]}")


## Check the current IDs in the dataset

In [ ]:
# ✅ Check current IDs in dataset_val
print(f"\n🔍 Total elements in 'colpo_val': {len(dataset_val)}")
print("\n📷 Examples from the 'colpo_val' dataset:")
for i, d in enumerate(dataset_val[:3]):
    nombre = Path(d["file_name"]).name
    id_expected = img_id_map.get(nombre, "❌ Not found")
    print(f"  [{i}] Current ID: {d['image_id']} | Expected: {id_expected} | File: {nombre}")
    for ann in d["annotations"]:
        print(f"    - Class: {ann['category_id']} | Segmentation: {str(ann['segmentation'])[:30]}...")


## Check that image IDs are unique

In [ ]:
# ✅ Step 2: Check that image IDs are unique in the dataset
ids_dataset = [d["image_id"] for d in dataset_val]
if len(set(ids_dataset)) == len(ids_dataset):
    print("\n✅ All 'image_id' values in 'colpo_val' are unique.")
else:
    print("\n❌ Duplicate 'image_id' values found in the dataset.")

## Load the exported COCO JSON file

In [ ]:
# ✅ Step 3: Load the exported JSON file (if it exists)
if COCO_JSON_PATH.exists():
    print(f"\n📂 File found: {COCO_JSON_PATH.name}")
    with open(COCO_JSON_PATH, 'r') as f:
        coco_data = json.load(f)

    ids_json = [img["id"] for img in coco_data.get("images", [])]
    print(f"📊 Total images in COCO JSON: {len(ids_json)}")


## Compare IDs between dataset and JSON

In [ ]:
    # ✅ Step 4: Compare image IDs between dataset and exported JSON
    print("\n🔍 Comparing 'image_id' between dataset and exported JSON...")
    ids_dataset_set = set(ids_dataset)
    ids_json_set = set(ids_json)

    missing_in_json = ids_dataset_set - ids_json_set
    missing_in_dataset = ids_json_set - ids_dataset_set

    if missing_in_json:
        print(f"⚠️ IDs present in the dataset but missing in the JSON: {sorted(missing_in_json)}")
    else:
        print("✅ All dataset IDs are present in the JSON.")

    if missing_in_dataset:
        print(f"⚠️ IDs present in the JSON but missing in the dataset: {sorted(missing_in_dataset)}")
    else:
        print("✅ All JSON IDs are present in the dataset.")
else:
    print(f"\n❌ File not found: {COCO_JSON_PATH}")
    print("   → Make sure it was properly generated during evaluation.")


## Regenerate the COCO JSON file with consistent image IDs

In [ ]:
# ✅ If IDs do not match, regenerate the COCO JSON file with consistent IDs
print("\n🔁 Regenerating COCO JSON file with consistent IDs...")

# You should already have this list loaded (anotaciones_val)
# If not available in this cell, load it from where it was previously generated
# anotaciones_val = [...]

json_coco_val = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "aw"},
        {"id": 2, "name": "m"},
        {"id": 3, "name": "p"},
        {"id": 4, "name": "av"},
        {"id": 5, "name": "y"}
    ]
}

import cv2
ann_id_counter = 1
for nombre, img_id in img_id_map.items():
    ruta = VAL_IMG_DIR / nombre
    if not ruta.exists():
        print(f"❌ Image not found: {nombre}")
        continue

    img = cv2.imread(str(ruta))
    if img is None:
        print(f"⚠️ Unable to read image: {nombre}")
        continue

    height, width = img.shape[:2]
    json_coco_val["images"].append({
        "id": img_id,
        "file_name": nombre,
        "width": width,
        "height": height
    })

    for ann in [a for a in anotaciones_val if a["nombre"] == nombre]:
        segmentacion = ann["segmentacion"]
        categoria_id = ann["categoria_id"]

        x_coords = segmentacion[0][0::2]
        y_coords = segmentacion[0][1::2]
        bbox = [
            float(min(x_coords)),
            float(min(y_coords)),
            float(max(x_coords) - min(x_coords)),
            float(max(y_coords) - min(y_coords))
        ]
        area = bbox[2] * bbox[3]

        json_coco_val["annotations"].append({
            "id": ann_id_counter,
            "image_id": img_id,
            "category_id": categoria_id,
            "segmentation": segmentacion,
            "bbox": bbox,
            "area": area,
            "iscrowd": 0
        })
        ann_id_counter += 1


## Save the updated COCO JSON file

In [ ]:
# Save the updated JSON
with open(COCO_JSON_PATH, "w") as f:
    json.dump(json_coco_val, f, indent=2)

print(f"\n✅ New COCO JSON file saved at: {COCO_JSON_PATH}")


## Perform inference and evaluation using COCO metrics

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# ⚙️ Create COCO evaluator
evaluator = COCOEvaluator("colpo_val", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "colpo_val")

# 🧠 Run inference and evaluation
results = inference_on_dataset(trainer.model, val_loader, evaluator)

# 📊 Display results
print("📊 Evaluation results:")
print(results)


## Evaluate segmentation performance using COCO metrics

In [ ]:
from detectron2.evaluation.coco_evaluation import instances_to_coco_json
from detectron2.data.datasets.coco import load_coco_json
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO

# ⚠️ Only if COCOEvaluator was used
coco_gt = COCO(str(COCO_JSON_PATH))
coco_dt = coco_gt.loadRes("output/colpo_model/coco_instances_results.json")

coco_eval = COCOeval(coco_gt, coco_dt, iouType="segm")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()


## Calculate and display average precision per class

In [ ]:
# Get AP per class
cats = coco_gt.loadCats(coco_gt.getCatIds())
class_names = [c["name"] for c in cats]
ap_per_class = coco_eval.eval["precision"]  # [T, R, K, A, M]
import numpy as np

ap_results = []
for idx, name in enumerate(class_names):
    precision = coco_eval.eval["precision"][:, :, idx, 0, 0]
    precision = precision[precision > -1]
    ap = np.mean(precision) if precision.size else float("nan")
    ap_results.append((name, ap))


## Visualize AP values as a heatmap

In [ ]:
# Display AP values as a table
df_ap = pd.DataFrame(ap_results, columns=["Class", "AP"]).dropna()
sns.heatmap(df_ap.set_index("Class").T, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("📊 AP per class (manually calculated)")
plt.show()


## Visualize random prediction examples from the validation dataset

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
import random
import cv2

# Test some visual predictions
for d in random.sample(dataset_val, 5):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1],
        MetadataCatalog.get("colpo_val"),
        scale=0.5,
        instance_mode=ColorMode.IMAGE_BW  # or use ColorMode.SEGMENTATION
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Prediction", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)


## Review the model configuration and metadata consistency

In [ ]:
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog

def revisar_configuracion(cfg):
    print("\n🧠 CONFIGURATION REVIEW")
    print("📦 Weights loaded from:", cfg.MODEL.WEIGHTS)
    print("📊 Number of classes:", cfg.MODEL.ROI_HEADS.NUM_CLASSES)
    print("📁 VAL dataset:", cfg.DATASETS.TEST)
    print("🎯 Score threshold:", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
    print("📎 Registered categories:", MetadataCatalog.get(cfg.DATASETS.TEST[0]).thing_classes)
    
    # Useful validations
    if cfg.MODEL.ROI_HEADS.NUM_CLASSES != len(MetadataCatalog.get(cfg.DATASETS.TEST[0]).thing_classes):
        print("❌ MISMATCH between NUM_CLASSES and registered categories. Please verify metadata registration.")
    else:
        print("✅ Number of classes and registered categories are aligned.")

# Run configuration review
revisar_configuracion(cfg)
